### Treinamento do Modelo Machine Learning

In [25]:
%pip install pandas scikit-learn xgboost joblib


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


#### Importações de Bibliotecas

In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import precision_score, accuracy_score
from typing import List, Tuple
from joblib import dump
import boto3
from io import StringIO
from sklearn.preprocessing import normalize
from sklearn.model_selection import GridSearchCV
from io import StringIO
from dotenv import load_dotenv
import os
load_dotenv()

True

#### Treinamento e Predições do Modelo XGBoost

#### Instruções

1. Renomear o arquivo .env_exemplo para somente .env
2. Adicionar popular as variaveis conforme o padrão de nomenclatura que voce utilizar

In [2]:
def read_from_s3(bucket_name:str):
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    response = []
    for obj in bucket.objects.all():
        body = obj.get()['Body'].read()
        s=str(body,'utf-8')
        data = StringIO(s)
        response.append(pd.read_csv(data, index_col=0))
    return response

In [3]:
bucket_name = os.environ['BUCKET_NAME']
csv_file_name = os.environ['CSV_FILE_NAME']
bucket_layer = os.environ['BUCKET_LAYER']
object_name = f'{bucket_layer}/{csv_file_name}'
df = read_from_s3(bucket_name)[0]
df.head()

,open,high,low,close,volume,edit_count,tomorrow,sentiment,neg_sentiment,fng_index,fng_class,fng_in_yest,fng_class_yest,target
2018-03-04,11497.400391,11512.599609,11136.099609,11512.599609,6084149760,2.933333,11573.299805,-0.258349,0.513519,44,Fear,56.0,Greed,1
2018-03-05,11532.400391,11704.099609,11443.900391,11573.299805,6468539904,2.866667,10779.900391,-0.259235,0.496852,55,Greed,44.0,Fear,0
2018-03-06,11500.099609,11500.099609,10694.299805,10779.900391,6832169984,2.666667,9965.570312,-0.249632,0.474630,59,Greed,55.0,Greed,0
2018-03-07,10803.900391,10929.500000,9692.120117,9965.570312,8797910016,2.633333,9395.009766,-0.216476,0.441296,37,Fear,59.0,Greed,0
2018-03-08,9951.440430,10147.400391,9335.870117,9395.009766,7186089984,2.600000,9337.549805,-0.188242,0.407963,39,Fear,37.0,Fear,0


In [4]:
# Definir preditores
# Estas são as features que estamos usando para prever o target
predictors: List[str] = [
    "close",
    "volume",
    "open",
    "high",
    "low",
    "edit_count",
    "sentiment",
    "neg_sentiment",
    "fng_in_yest",
    "fng_class_yest_adjusted"
]

# Função de predição
def predict(train: pd.DataFrame, test: pd.DataFrame, predictors: List[str], model: XGBClassifier) -> pd.DataFrame:
    model.fit(train[predictors], train["target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="predictions")
    return pd.concat([test["target"], preds], axis=1)

# Função de backtesting
# Usaremos uma abordagem de janela móvel para fazer o backtesting do modelo.
# Pegaremos uma janela de dados, usaremos para treinar o modelo e, em seguida, usaremos o modelo para prever a próxima janela de dados.
# Repetiremos esse processo até termos usado todos os dados.
# O parâmetro start significa 3 anos de dados para treinar o modelo.
# O parâmetro step significa 150 dias de dados para testar o modelo.
def backtest(data: pd.DataFrame, model: XGBClassifier, predictors: List[str], start: int = 1095, step: int = 150) -> pd.DataFrame:
    all_predictions = []

    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i + step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)

    return pd.concat(all_predictions)

# Função de avaliação do modelo
def evaluate_model(predictions: pd.DataFrame) -> Tuple[float, float]:
    precision = precision_score(predictions["target"], predictions["predictions"])
    accuracy = accuracy_score(predictions["target"], predictions["predictions"])
    return precision, accuracy

In [5]:
# Carregar os dados do arquivo CSV
data = read_from_s3(bucket_name)[0]
#Adjusting fng classification values
map = {'Neutral': 0, 'Greed': 1, 'Fear': -1, 'Extreme Fear': -2, 'Extreme Greed': 2}
data['fng_class_yest_adjusted'] = data['fng_class_yest'].map(map)
data.drop(columns=['fng_class', 'fng_class_yest'], axis=1, inplace=True)
data.head()


,open,high,low,close,volume,edit_count,tomorrow,sentiment,neg_sentiment,fng_index,fng_in_yest,target,fng_class_yest_adjusted
2018-03-04,11497.400391,11512.599609,11136.099609,11512.599609,6084149760,2.933333,11573.299805,-0.258349,0.513519,44,56.0,1,1
2018-03-05,11532.400391,11704.099609,11443.900391,11573.299805,6468539904,2.866667,10779.900391,-0.259235,0.496852,55,44.0,0,-1
2018-03-06,11500.099609,11500.099609,10694.299805,10779.900391,6832169984,2.666667,9965.570312,-0.249632,0.474630,59,55.0,0,1
2018-03-07,10803.900391,10929.500000,9692.120117,9965.570312,8797910016,2.633333,9395.009766,-0.216476,0.441296,37,59.0,0,1
2018-03-08,9951.440430,10147.400391,9335.870117,9395.009766,7186089984,2.600000,9337.549805,-0.188242,0.407963,39,37.0,0,-1


In [14]:
predictors: List[str] = [
    "close",
    "volume",
    "open",
    "high",
    "low",
    "fng_in_yest",
]

In [21]:
# Criar e treinar o modelo XGBoost
model = XGBClassifier(random_state=1, learning_rate=0.3, n_estimators=200, colsample_bytree = 0.5, max_depth = 5)
predictions = backtest(data, model, predictors)

# Avaliar o modelo
precision, accuracy = evaluate_model(predictions)
print(f"Precisão do modelo: {precision:.2f}")
print(f"Acurácia do modelo: {accuracy:.2f}")
print(predictions)

Precisão do modelo: 0.50
Acurácia do modelo: 0.50
            target  predictions
2021-03-06       1            0
2021-03-07       1            0
2021-03-08       1            0
2021-03-09       1            0
2021-03-10       1            0
...            ...          ...
2024-09-01       1            1
2024-09-02       0            0
2024-09-03       1            0
2024-09-04       0            0
2024-09-05       0            1

[1280 rows x 2 columns]


In [102]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2378 entries, 2018-03-02 to 2024-09-06
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   open                         2378 non-null   float64
 1   high                         2378 non-null   float64
 2   low                          2378 non-null   float64
 3   close                        2378 non-null   float64
 4   volume                       2378 non-null   int64  
 5   edit_count                   2378 non-null   float64
 6   tomorrow                     2377 non-null   float64
 7   sentiment                    2378 non-null   float64
 8   neg_sentiment                2378 non-null   float64
 9   fng_index                    2378 non-null   int64  
 10  target                       2378 non-null   int64  
 11  fng_classification_adjusted  2378 non-null   int64  
dtypes: float64(8), int64(4)
memory usage: 241.5+ KB


#### Diferent models try

In [132]:
def predict_norm(train: pd.DataFrame, test: pd.DataFrame, predictors: List[str], model: XGBClassifier) -> pd.DataFrame:
    X_train = normalize(train[predictors])
    y_train = train["target"]
    model.fit(X_train, y_train)
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="predictions")
    return pd.concat([test["target"], preds], axis=1)

def backtest_norm(data: pd.DataFrame, model: XGBClassifier, predictors: List[str], start: int = 1095, step: int = 150) -> pd.DataFrame:
    all_predictions = []

    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i + step)].copy()
        predictions = predict_norm(train, test, predictors, model)
        all_predictions.append(predictions)

    return pd.concat(all_predictions)

In [112]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [115]:
train = data.iloc[0:2000]
test = data.iloc[2000:2378]

## prepare data for models
X_train = normalize(train[predictors])
y_train = train["target"]

In [119]:
def run_grid_search_cv(model, params, X, y):
    search_cv = GridSearchCV(model, params)
    search_cv.fit(X, y)
    return search_cv.best_estimator_


In [120]:
rf_params = {
    'n_estimators':[100, 125, 150, 200],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [3, 5, 7, 9]
}
model = RandomForestClassifier()
rf_best = run_grid_search_cv(model=model, params=rf_params, X=X_train, y=y_train)

c:\Users\I857413\Desktop\TechSkills\Forks\bitcoin_trend_prediction\.venv\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [137]:
predictions = backtest_norm(data, rf_best, predictors)

# Avaliar o modelo
precision, accuracy = evaluate_model(predictions)
print(f"Precisão do modelo: {precision:.2f}")
print(f"Acurácia do modelo: {accuracy:.2f}")
print(predictions)

c:\Users\I857413\Desktop\TechSkills\Forks\bitcoin_trend_prediction\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
c:\Users\I857413\Desktop\TechSkills\Forks\bitcoin_trend_prediction\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
c:\Users\I857413\Desktop\TechSkills\Forks\bitcoin_trend_prediction\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
c:\Users\I857413\Desktop\TechSkills\Forks\bitcoin_trend_prediction\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
c:\Users\I857413\Desktop\TechSkills\Forks\bitcoin_trend_prediction\.venv\Lib\site-packages\sklearn\base.py:486: User

Precisão do modelo: 0.49
Acurácia do modelo: 0.50
            target  predictions
2021-03-04       1            1
2021-03-05       0            1
2021-03-06       1            1
2021-03-07       1            1
2021-03-08       1            1
...            ...          ...
2024-09-02       0            0
2024-09-03       1            0
2024-09-04       0            0
2024-09-05       0            0
2024-09-06       0            0

[1283 rows x 2 columns]


c:\Users\I857413\Desktop\TechSkills\Forks\bitcoin_trend_prediction\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
c:\Users\I857413\Desktop\TechSkills\Forks\bitcoin_trend_prediction\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


In [130]:
knn_params = {
    'n_neighbors':[9,10,11,12,13,14],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'weights': ['uniform', 'distance']
}
model = KNeighborsClassifier()
knn_best = run_grid_search_cv(model=model, params=knn_params, X=X_train, y=y_train)
print(knn_best)

KNeighborsClassifier(n_neighbors=9)


In [138]:
predictions = backtest_norm(data, knn_best, predictors)

# Avaliar o modelo
precision, accuracy = evaluate_model(predictions)
print(f"Precisão do modelo: {precision:.2f}")
print(f"Acurácia do modelo: {accuracy:.2f}")
print(predictions)

Precisão do modelo: 0.50
Acurácia do modelo: 0.50
            target  predictions
2021-03-04       1            0
2021-03-05       0            0
2021-03-06       1            0
2021-03-07       1            0
2021-03-08       1            0
...            ...          ...
2024-09-02       0            0
2024-09-03       1            0
2024-09-04       0            0
2024-09-05       0            0
2024-09-06       0            0

[1283 rows x 2 columns]


c:\Users\I857413\Desktop\TechSkills\Forks\bitcoin_trend_prediction\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
c:\Users\I857413\Desktop\TechSkills\Forks\bitcoin_trend_prediction\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
c:\Users\I857413\Desktop\TechSkills\Forks\bitcoin_trend_prediction\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
c:\Users\I857413\Desktop\TechSkills\Forks\bitcoin_trend_prediction\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
c:\Users\I857413\Desktop\TechSkills\Forks\bitcoin_trend_prediction\.venv\Lib\site-packages\sklearn\base.py:486: User

#### Serialização do Modelo

In [28]:
# Salvar o modelo treinado em um arquivo
model_filename = "btc_trend_prediction_model.joblib"
dump(model, model_filename)

['btc_trend_prediction_model.joblib']